In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append('../../')

from environment.env import Environment, EnvironmentConfig
from environment.utils import read_ply, create_pcd, reconstruct_features

## Load and create an environment

We do this by directing it to a file path for an image/video

In [2]:
spec = 'rats'
base_path = f'/workspace/fieldwork-data/{spec}/2024-07-11'
file_path = f"{base_path}/SplatsSD/C0119.MP4"

# We set our methods here --> we can use splatting for 3D data and clip for 2D data
config = EnvironmentConfig(
    file_path=file_path,
    dtype='3d',
    method='feature-splatting'
)

# Create the environment
env = Environment(config)

Next run preprocessing. For 3d data, this runs ```ns-process-data``` to decimate a video into images, then runs COLMAP to define structure from motion.

In [ ]:
env.preprocess_data()

Extract features using the current method --> for 3d data, this will create gaussian splats/nerfs using the specified method

In [ ]:
env.extract_features()

### Test exporting from a trained model

In [ ]:
from nerfstudio.models.splatfacto import SplatfactoModel
from nerfstudio.pipelines.base_pipeline import Pipeline, VanillaPipeline
from nerfstudio.utils.eval_utils import eval_setup

In [6]:
import glob
from pathlib import Path
config_fns = glob.glob(os.path.join(env.config['model_path'], '**/config.yml'), recursive=True)
load_config = Path(config_fns[0])

Load the pipeline and model

In [ ]:
_, pipeline, _, _ = eval_setup(load_config, test_mode="inference")

assert isinstance(pipeline.model, SplatfactoModel)

model: SplatfactoModel = pipeline.model

Get parameters of gaussians

In [ ]:
params = model.gauss_params
reconstructed_features = reconstruct_features(model, params['distill_features'])

### Test manipulation of ply file

In [ ]:
env.config

In [ ]:
ply_fn = env.config['output_path'] / 'exports' / 'splat.ply'
ply_info = read_ply(ply_fn)
pcd = create_pcd(ply_info)


In [16]:
import numpy as np
import math 

def align_pcd_floor(pcd, dist_threshold=0.02, visualize=False):
    
    # Uses RANSAC to sample points, finding plane with largest support
    # given sampling
    floor = get_floor_plane(pcd, dist_threshold=dist_threshold, visualize=visualize)
    a, b, c, d = floor

    # Normalize the normal vector
    normal = np.array([a, b, c])
    normal /= np.linalg.norm(normal)
    
    # Ensure normal points upward
    if normal[2] < 0:
        normal = -normal

    # Compute rotation to align the normal with Z-axis
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(normal, z_axis)
    rotation_angle = np.arccos(np.clip(np.dot(normal, z_axis), -1.0, 1.0))

    if np.linalg.norm(rotation_axis) < 1e-6:
        R = np.eye(3)
    else:
        rotation_axis /= np.linalg.norm(rotation_axis)
        axis_angle = rotation_axis * rotation_angle
        R = pcd.get_rotation_matrix_from_axis_angle(axis_angle)

    # Rotate first (no center -> about origin)
    pcd.rotate(R, center=(0, 0, 0))

    # Recompute floor plane after rotation
    new_plane = get_floor_plane(pcd, dist_threshold=dist_threshold, visualize=visualize)
    _, _, _, d_new = new_plane

    # Translate the floor to z=0 (d is distance from origin along Z after alignment)
    pcd.translate((0, 0, -d_new))  # since now normal ≈ [0,0,1], use d_new directly

def get_floor_plane(pcd, dist_threshold=0.02, visualize=False):
    plane_model, inliers = pcd.segment_plane(distance_threshold=dist_threshold,
                                             ransac_n=3,
                                             num_iterations=1000)
    return plane_model

In [17]:
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02)

cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=20, radius=0.05)

align_pcd_floor(cl)

In [32]:
import open3d as o3d

# align_pcd_floor(cl)
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(cl, 0.02)
mesh.compute_vertex_normals()

# Convert mesh and point cloud vertices to numpy arrays
mesh_vertices = np.asarray(mesh.vertices)
pcd_points = np.asarray(cl.points)
pcd_colors = np.asarray(cl.colors)

# Find nearest neighbor in point cloud for each vertex in the mesh
pcd_kd_tree = o3d.geometry.KDTreeFlann(cl)
vertex_colors = []

for vertex in mesh_vertices:
    _, idx, _ = pcd_kd_tree.search_knn_vector_3d(vertex, 1)
    vertex_colors.append(pcd_colors[idx[0]])

mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)


# cl.estimate_normals()
# radii = [0.005, 0.01, 0.02, 0.04]
# rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd, o3d.utility.DoubleVector(radii))

# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     cl, o3d.utility.DoubleVector(radii))

In [ ]:
o3d.visualization.draw_plotly([mesh], point_sample_factor=0.5)

In [ ]:
import open3d as o3d

o3d.visualization.draw_plotly([cl], point_sample_factor=0.5)